# Perform Multi-Table Synthesization

In this exercise, we are going to walk through the synthesis of a relational multi-table structure. For that, we will be using the Berka dataset [[1](#refs)]: a dataset containing Czech bank transactions. It consists of a total of 8 tables, whereas one of these ("district") only serves as a basic reference table, while all others are considered to be privacy-sensitive and thus are being synthesized.

<img src='https://raw.githubusercontent.com/mostly-ai/mostly-tutorials/dev/multi-table/berka-original.png' width="600px"/>

## Data preparation

Fetch original data

In [ ]:
import pandas as pd

base_url = "https://github.com/mostly-ai/public-demo-data/raw/dev/berka/data/"
originals = {
    "client": pd.read_csv(base_url + "client.csv.gz", low_memory=False),
    "disposition": pd.read_csv(base_url + "disp.csv.gz", low_memory=False),
    "card": pd.read_csv(base_url + "card.csv.gz", low_memory=False),
    "account": pd.read_csv(base_url + "account.csv.gz", low_memory=False),
    "transaction": pd.read_csv(base_url + "trans.csv.gz", low_memory=False),
    "loan": pd.read_csv(base_url + "loan.csv.gz", low_memory=False),
    "order": pd.read_csv(base_url + "order.csv.gz", low_memory=False),
}

originals['account']['date'] = pd.to_datetime(originals['account']['date'])
originals['transaction']['date'] = pd.to_datetime(originals['transaction']['date'])

In [ ]:
for k in originals:
    print("===", k, "===")
    display(originals[k].sample(n=3))

## Train a multi-table generator

Configuring a mutli-table generator is simply done, by configuring each table, their primary key as well as all their foreign key relations.

As can be seen from the entity relationship diagram above, the Berka dataset has two privacy-sensitive tables, that do not reference any other privacy-sensitive table. These two top-level tables ("client" and "account") will act as our subject tables, that are independently generated. All other tables are linked to these, and thus will also then be generated in their context.

Additionally, it can be seen, that the two subject tables share the same child table named "disposition". MOSTLY AI supports retaining the correlations between a child and its parent table (as well as to its great-parent and parent-sibling), but this can only be configured for a single parent (the so called "context" parent). We therefore have to decide which of these two foreign key relationships is then considered as context, and which not. Only for the former correlations can be retained across tables, while for the latter the parents will get randomly assigned (while still retaining referential integrity).

<img src='https://raw.githubusercontent.com/mostly-ai/mostly-tutorials/dev/multi-table/berka-configuration.png' width="600px"/>

In [ ]:
from mostlyai.sdk import MostlyAI

# initialize SDK
mostly = MostlyAI(local=True)  # or: MostlyAI(base_url='xxx', api_key='xxx')

config = {
    "name": "Multi-table Tutorial - Berka",
    "tables": [
        {
            "name": "client", 
            "data": originals["client"],
            "primary_key": "client_id"
        },
        {
            "name": "disposition",
            "data": originals["disposition"],
            "primary_key": "disp_id",
            "foreign_keys": [
                {"column": "account_id", "referenced_table": "account", "is_context": True},
                {"column": "client_id", "referenced_table": "client", "is_context": False}
            ]
        },
        {
            "name": "card",
            "data": originals["card"],
            "primary_key": "card_id",
            "foreign_keys": [{"column": "disp_id", "referenced_table": "disposition", "is_context": True}]
        },
        {
            "name": "account", 
            "data": originals["account"],
            "primary_key": "account_id"
        },
        {
            "name": "transaction",
            "data": originals["transaction"],
            "primary_key": "trans_id",
            "foreign_keys": [{"column": "account_id", "referenced_table": "account", "is_context": True}]
        },
        {
            "name": "loan",
            "data": originals["loan"],
            "primary_key": "loan_id",
            "foreign_keys": [{"column": "account_id", "referenced_table": "account", "is_context": True}]
        },
        {
            "name": "order",
            "data": originals["order"],
            "primary_key": "order_id",
            "foreign_keys": [{"column": "account_id", "referenced_table": "account", "is_context": True}]
        }
    ]
}

# configure a generator, but don't yet start the training thereof
g = mostly.train(config=config, start=False)

# open generator in a new browser tab
g.open()

You can now also inspect the configuration on the Web UI. It will look like this:

<img src='https://raw.githubusercontent.com/mostly-ai/mostly-tutorials/dev/multi-table/berka-ui-1.png' width="300px"/>
<img src='https://raw.githubusercontent.com/mostly-ai/mostly-tutorials/dev/multi-table/berka-ui-2.png' width="300px"/>

Now, launch the training, and wait for it to be finished. This shouldn't take longer than 10 minutes.

In [ ]:
g.training.start()
g = g.training.wait(progress_bar=True)

## Generate a multi-table dataset

Once, the training has completed, you can generate a multi-table dataset with. If you do not specify the sample size, then the platform will generate as many subject records, as there were in the original subject tables. Otherwise, you will need to specify for each subject table, the number of records, as these are independently sampled.

In [ ]:
# generate a synthetic dataset
sd = mostly.generate(g, size={'account': 4500, 'client': 5369}, start=False)
sd.open()

In [ ]:
sd.generation.start()
sd = sd.generation.wait(progress_bar=True)

In [ ]:
# once done, fetch the synthetic data as dictionary of DataFrames
synthetics = sd.data()

### Show sample records for each table

In [ ]:
for k in synthetics:
    print("===", k, "===")
    display(synthetics[k].sample(n=3))

### Check basic statistics

The newly generated tables are statistically representative of the original.

In [ ]:
display(synthetics['transaction']['amount'].quantile(q=[.1, .5, .9]))
display(originals['transaction']['amount'].quantile(q=[.1, .5, .9]))

In [ ]:
display(synthetics['transaction']['date'].quantile(q=[.1, .5, .9], interpolation='nearest'))
display(pd.to_datetime(originals['transaction']['date']).quantile(q=[.1, .5, .9], interpolation='nearest'))

In [ ]:
import matplotlib.pyplot as plt

# Convert the 'date' column to datetime format for the third dataframe
originals['transaction']['date'] = pd.to_datetime(originals['transaction']['date'])
synthetics['transaction']['date'] = pd.to_datetime(synthetics['transaction']['date'])

# Create the side-by-side scatter plots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6), sharey=True)

# Plot original data
axes[0].scatter(originals['transaction']['date'], originals['transaction']['amount'], s=1, alpha=0.5)
axes[0].set_title('Amount vs Date (Original)')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Amount')
axes[0].grid(True)

# Plot synthetic data
axes[1].scatter(synthetics['transaction']['date'], synthetics['transaction']['amount'], s=1, alpha=0.5)
axes[1].set_title('Amount vs Date (MOSTLY AI)')
axes[1].set_xlabel('Date')
axes[1].grid(True)

# Adjust layout
plt.tight_layout()

# show
plt.show()

### Check referential integrity

The newly generated foreign keys are also present as primary keys in the connected tables.

In [ ]:
assert synthetics['transaction']['account_id'].isin(synthetics['account']['account_id']).all()
assert synthetics['card']['disp_id'].isin(synthetics['disposition']['disp_id']).all()

### Check context FK relations

The cardinality of context FK relations is perfectly retained.

In [ ]:
print('Orders per Account - Synthetic')
display(synthetics['order'].groupby('account_id').size().value_counts())
print('\nOrders per Account - Original')
display(originals['order'].groupby('account_id').size().value_counts())

In [ ]:
print('Cards per Disposition - Synthetic')
display(synthetics['card'].groupby('disp_id').size().value_counts())
print('\nCards per Disposition - Original')
display(originals['card'].groupby('disp_id').size().value_counts())

## References<a class="anchor" name="refs"></a>

1. https://data.world/lpetrocelli/czech-financial-dataset-real-anonymized-transactions